In [1]:
import os
import pysam
import pandas as pd
import numpy as np

In [2]:
os.chdir("/gscratch/stergachislab/bmallo/large_home/fiberseq-smk/results/LDLR_sc_control")

### Open the bam file to read from, and create a new bam file to write to

In [3]:
bam = pysam.AlignmentFile('aligned.fiberseq.bam', 'rb')

chrom_wt_reads = pysam.AlignmentFile('chromatinized_wt_reads.bam', 'wb', template=bam)

#### Set variable containing WT LDLR promoter sequence

In [4]:
LDLR_promoter_seq = 'AGCTCTTCACCGGAGACCCAAATACAACAAATCAAGTCGCCTGCCCTGGCGACACTTTCGAAGGACTGGAGTGGGAATCAGAGCTTCACGGGTTAAAAAGCCGATGTCACATCGGCCGTTCGAAACTCCTCCTCTTGCAGTGAGGTGAAGACATTTGAAAATCACCCCACTGCAAACTCCTCCCCCTGCTAGAAACCTCACATTGAAATGCTGTAAATGACGTGGGCCCCGAGTGCAATCGCGGGAAGCCAGGGTTTCCAGCTAGGACACAGCAGGTCGTGATCCGGGTCGGGACACTGCCTGGCAGAGGCTGCGAGC'

### Filter out full length LDLR reads that contain nucleosome calls, and write these reads to the new bam file

In [5]:
pysam.AlignmentFile.count(bam)

3125

In [14]:
iter = bam.fetch('corrected_linear_LDLR_kpnI')

seq_count = 0
full_seq_list = []
wt_nuc_count = 0 

for record in iter:

    if not record.has_tag('ns'): continue

    seq_count += 1

    read = record.query_sequence

    read_length = len(read)

    if (4740 < read_length) and (read_length < 4760):

        full_seq_list.append(read)

        if LDLR_promoter_seq in read:

            wt_nuc_count += 1 

            chrom_wt_reads.write(record)


print(f'Total reads = {seq_count} reads')
print(f'Full length reads = {len(full_seq_list)} reads')
print(f'Full length chromatinized reads = {wt_nuc_count} reads')
chrom_wt_reads.close()


Total reads = 663 reads
Full length reads = 100 reads
Full length chromatinized reads = 13 reads


In [11]:
import pysam
import numpy as np
LDLR_bam = pysam.AlignmentFile('aligned.fiberseq.bam', 'rb')

seq_list = []
promoter_list = []

for read in LDLR_bam:

	if read.is_unmapped: continue 

	seq = read.query_sequence

	if 4740 < len(seq) < 4760:

		seq_list.append(seq)

		aligned_pairs = np.array(read.get_aligned_pairs(matches_only=True)).transpose().astype(int)

		seq_start = aligned_pairs[0,aligned_pairs[1] == 2659]
		seq_stop = aligned_pairs[0, aligned_pairs[1] == 2718]
   
		if (len(seq_start) == 1 and len(seq_stop) == 1):

			promoter_list.append(seq[int(seq_start):int(seq_stop)])
		
promoter_set = set(promoter_list)

TabError: inconsistent use of tabs and spaces in indentation (2876418023.py, line 22)